Bronze Layer

In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
# Read raw CSV from Bronze
df_bronze = spark.read.csv("Files/Financial_Sample.csv", header=True, inferSchema=True)

# Show first 5 rows
#df_bronze.show(5)

#Files/Financial_Sample.csv


StatementMeta(, d82248a4-6c30-4fcd-8170-767a826f8f93, 4, Finished, Available, Finished)

Silver Layer

In [3]:
df_bronze_cleaned = df_bronze.toDF(*[c.strip() for c in df_bronze.columns])

StatementMeta(, d82248a4-6c30-4fcd-8170-767a826f8f93, 5, Finished, Available, Finished)

In [4]:
from pyspark.sql.functions import regexp_replace

numeric_cols = [
    "Manufacturing Price", "Sale Price", "Gross Sales",
    "Discounts", "Sales", "COGS", "Profit"
]

df_numeric_cleaned = df_bronze_cleaned
for col_name in numeric_cols:
    df_numeric_cleaned = df_numeric_cleaned.withColumn(
        col_name,
        regexp_replace(col(col_name), "[$,]", "")  # remove $ and ,
    )

StatementMeta(, d82248a4-6c30-4fcd-8170-767a826f8f93, 6, Finished, Available, Finished)

In [5]:
from pyspark.sql.types import IntegerType, DecimalType, DateType
from pyspark.sql.functions import to_date, col

df_silver = df_numeric_cleaned.select(
    col("Segment").alias("segment"),
    col("Country").alias("country"),
    col("Product").alias("product"),
    col("Discount Band").alias("discount_band"),
    col("Units Sold").cast(IntegerType()).alias("units_sold"),
    col("Manufacturing Price").cast(DecimalType(10,2)).alias("manufacturing_price"),
    col("Sale Price").cast(DecimalType(10,2)).alias("sale_price"),
    col("Gross Sales").cast(DecimalType(12,2)).alias("gross_sales"),
    col("Discounts").cast(DecimalType(12,2)).alias("discounts"),
    col("Sales").cast(DecimalType(12,2)).alias("sales"),
    col("COGS").cast(DecimalType(12,2)).alias("cogs"),
    col("Profit").cast(DecimalType(12,2)).alias("profit"),
    to_date(col("Date"), "M/d/yyyy").alias("date"),
    col("Month Number").cast(IntegerType()).alias("month_number"),
    col("Month Name").alias("month_name"),
    col("Year").cast(IntegerType()).alias("year"),
    col("date").alias("date_hierarchy")
)

df_silver.show(5)

StatementMeta(, d82248a4-6c30-4fcd-8170-767a826f8f93, 7, Finished, Available, Finished)

+----------+-------+-----------+-------------+----------+-------------------+----------+-----------+---------+--------+--------+--------+----------+------------+----------+----+--------------+
|   segment|country|    product|discount_band|units_sold|manufacturing_price|sale_price|gross_sales|discounts|   sales|    cogs|  profit|      date|month_number|month_name|year|date_hierarchy|
+----------+-------+-----------+-------------+----------+-------------------+----------+-----------+---------+--------+--------+--------+----------+------------+----------+----+--------------+
|Government| Canada| Carretera |        None |      1618|               3.00|     20.00|   32370.00|     NULL|32370.00|16185.00|16185.00|2014-01-01|           1|  January |2014|      1/1/2014|
|Government|Germany| Carretera |        None |      1321|               3.00|     20.00|   26420.00|     NULL|26420.00|13210.00|13210.00|2014-01-01|           1|  January |2014|      1/1/2014|
| Midmarket| France| Carretera |   

## Gold Layer — Aggregation & Business Metrics

In [14]:
#from pyspark.sql.functions import quarter

# Add quarter based on the date column
#df_gold = df_silver.withColumn("quarter", quarter(col("date")))

StatementMeta(, 8201ba2e-47fb-4176-b7f9-1614b4510716, 17, Finished, Available, Finished)

In [7]:
from pyspark.sql.functions import (
    sum, round, col, month, year, quarter, concat, lit, date_format
)

# Step 1: Add date components in the Silver layer itself
df_silver_time = df_silver.withColumn("year", year(col("date")))\
                          .withColumn("month_number", month(col("date")))\
                          .withColumn("quarter_number", quarter(col("date")))\
                          .withColumn("quarter_label", concat(lit("Q"), col("quarter_number")))\
                          .withColumn("month_name_short", date_format(col("date"), "MMM"))

# Step 2: Aggregate metrics to create Gold layer
df_gold_time = df_silver_time.groupBy(
                    "year", "quarter_label", "month_number", "month_name_short",
                    "country", "segment", "product", "date"
                )\
                .agg(
                    round(sum("units_sold"), 0).alias("total_units"),
                    round(sum("sales"), 2).alias("total_sales"),
                    round(sum("profit"), 2).alias("total_profit"),
                    round(sum("discounts"), 2).alias("total_discounts"),
                    round((sum("profit") / sum("sales")) * 100, 2).alias("profit_margin_percentage")
                )

# Step 3: Save as Gold table
df_gold_time.write.option("overwriteSchema", "true").mode("overwrite").saveAsTable("sales_data_gold_time")
#df_gold_time.write.mode("overwrite").saveAsTable("sales_data_gold_time")

# Step 4: Verify sample output
df_gold_time.show(10)

StatementMeta(, d82248a4-6c30-4fcd-8170-767a826f8f93, 9, Finished, Available, Finished)

+----+-------------+------------+----------------+--------------------+--------------+-----------+----------+-----------+-----------+------------+---------------+------------------------+
|year|quarter_label|month_number|month_name_short|             country|       segment|    product|      date|total_units|total_sales|total_profit|total_discounts|profit_margin_percentage|
+----+-------------+------------+----------------+--------------------+--------------+-----------+----------+-----------+-----------+------------+---------------+------------------------+
|2014|           Q2|           5|             May|              France|    Government|     Paseo |2014-05-01|       1030|    7137.90|     1987.90|          72.10|                   27.85|
|2013|           Q3|           9|             Sep|United States of ...|    Government|       VTT |2013-09-01|        349|  117264.00|    26524.00|        4886.00|                   22.62|
|2013|           Q4|          12|             Dec|United Sta